In [1]:
from keras.datasets import mnist
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Conv2DTranspose, UpSampling2D
import numpy as np
from keras.constraints import max_norm
from sklearn.model_selection import train_test_split

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
noise_factor = 0.25
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0. , 1.)
x_test_noisy = np.clip(x_test_noisy, 0. , 1.)

In [8]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train_noisy = x_train_noisy.reshape(x_train_noisy.shape[0], 28, 28, 1)
x_test_noisy = x_test_noisy.reshape(x_test_noisy.shape[0], 28, 28, 1)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_noisy = x_train_noisy.astype('float32')
x_test_noisy = x_test_noisy.astype('float32')

In [10]:
x_train /= 255
x_test /= 255
x_train_noisy /= 255
x_test_noisy /= 255

In [11]:
# Convolutional Autoencoder Neural Network

# Encoding the image
auto_encoder = Sequential()
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)))
auto_encoder.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
auto_encoder.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

# Reconstructing the image
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
auto_encoder.add(UpSampling2D((2, 2)))
auto_encoder.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
auto_encoder.add(UpSampling2D((2, 2)))
auto_encoder.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

In [12]:
auto_encoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
auto_encoder.fit(x_train_noisy, x_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 63s 33ms/step - loss: 0.0287 - accuracy: 0.4036
Epoch 2/5
1875/1875 [==============================] - 68s 36ms/step - loss: 5.5374e-05 - accuracy: 0.4041
Epoch 3/5
1875/1875 [==============================] - 376s 201ms/step - loss: 5.5072e-05 - accuracy: 0.4042
Epoch 4/5
1875/1875 [==============================] - 118s 63ms/step - loss: 5.4976e-05 - accuracy: 0.4042
Epoch 5/5
1875/1875 [==============================] - 96s 51ms/step - loss: 5.4929e-05 - accuracy: 0.4042


In [13]:
# Removing the noice through auto encoding of the noisy data

x_train_noice_reduced = auto_encoder.predict(x_train_noisy)
x_test_noice_reduced = auto_encoder.predict(x_test_noisy)

In [14]:
# Model used in 1 and 2
model = Sequential()
model.add(Conv2D(28, activation='relu', kernel_size=(3,3), input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50,activation='relu'))
model.add(Dense(10,activation='softmax'))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train_noice_reduced,y=y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 2.3019 - accuracy: 0.1093
Epoch 2/10
1875/1875 [==============================] - 12s 7ms/step - loss: 2.3014 - accuracy: 0.1126
Epoch 3/10
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3013 - accuracy: 0.1129
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3011 - accuracy: 0.1137
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3013 - accuracy: 0.1122
Epoch 6/10
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3015 - accuracy: 0.1093
Epoch 7/10
1875/1875 [==============================] - 12s 7ms/step - loss: 2.3015 - accuracy: 0.1116
Epoch 8/10
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3012 - accuracy: 0.1109
Epoch 9/10
1875/1875 [==============================] - 12s 6ms/step - loss: 2.3016 - accuracy: 0.1111
Epoch 10/10
1875/1875 [==============================] - 12s 6ms/step - l

In [15]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.3013 - accuracy: 0.1135


[2.301305055618286, 0.11349999904632568]